In [ ]:
<a href="https://colab.research.google.com/github/ryskks/usedicom/blob/main/multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 複数のスライス画像を扱う